In [21]:
from linkedin_api import Linkedin
from concurrent.futures import ThreadPoolExecutor
import sqlite3
from datetime import datetime

# Save in a file called creds.txt on the same directory as this script your email (first line) and linkedin password (second line)
f = open("creds.txt")
creds = f.readlines()
f.close()

email = creds[0].split('\n')[0]
password = creds[1].split('\n')[0]

api = Linkedin(email, password)


def create_table():
    """Create database only if it does not exist"""

    conn = sqlite3.connect("requested.db")
    c = conn.cursor()

    c.execute('''
          CREATE TABLE IF NOT EXISTS requested(
            [ID] INTEGER PRIMARY KEY, 
            [LinkedIn_username] TEXT,
            [Job title] TEXT,
            [Request sent on] TEXT)
          ''')      
    conn.commit()

    c.execute('''
          CREATE TABLE IF NOT EXISTS prev_queries(
            [ID] INTEGER PRIMARY KEY,
            [query] TEXT,
            [depth] TEXT
          )''' )
    conn.commit()

    conn.close()


def insert_entry(username, job_title, date):
    """Insert contact into database"""

    conn = sqlite3.connect("requested.db")
    c = conn.cursor()

    c.execute(f'''
          INSERT INTO requested(
            [LinkedIn_username],
            [Job title],
            [Request sent on])
          VALUES(
            '{username}',
            '{job_title}',
            '{date}'
          )
          ''')  
    conn.commit()
    conn.close()


def insert_query(query, depth):
    """Insert contact into database"""

    conn = sqlite3.connect("requested.db")
    c = conn.cursor()

    c.execute(f'''
          INSERT INTO prev_queries(
            [query],
            [depth])
          VALUES(
            '{query}',
            '{depth}'
          )
          ''')  
    conn.commit()
    conn.close()



def check_user(username):
    """Check if a contact is already in database"""

    conn = sqlite3.connect("requested.db")
    c = conn.cursor()

    c.execute(f'''
         SELECT * FROM requested WHERE Linkedin_username = '{username}';
          ''')
                     
    contact = c.fetchall()
    conn.commit()
    conn.close()
    return contact


create_table()

def send_contact_request(contact, percent):
    """Function to send contact request if it's not already on the DB"""
    check = check_user(contact[0])
    if len(check) == 0:
        try: 
            Linkedin.add_connection(api,contact[0])
            print(percent,"%  ::  ","Request sent to: ", contact)
            insert_entry(contact[0], contact[1], datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        except:
            pass
    else:
        print(f"{percent}%  ::  Contact {contact[0]} was already sent a connection request on {check[0][3]}")
    

## Search people by keywords
To add more people to the search include in network_depths more values  of: "F" (first order), "S" (second order), "O" (third and other orders)

In [26]:

keywords = ["security", "hiring", "meta"]

keywords_str = " ".join(keywords).strip()
depths = ["F","S"]
depths_str = ''.join(depths)
people = Linkedin.search_people(api, keywords={keywords_str}, network_depths=depths)

In the next block "threshold" is the minimum number of keywords that the job title of the person needs to have in order to send him/her a request

In [27]:

threshold = 1
contact_list = []

for person in people:
    if person["jobtitle"] != None:
        count = 0
        add_contact = False
        for keyword in keywords:
            if keyword.lower() in person["jobtitle"].lower():
                count += 1
            if count == threshold:
                add_contact = True
                break
        if add_contact:
            contact_list.append((person["public_id"], person["jobtitle"]))

print(len(contact_list), "requests will be sent")


3 requests will be sent


The next block isn't necessary, only serves to check which recruiters will be contacted

In [ ]:
for contact in contact_list:
    print(contact)

# Send requests
Send requests to selected people. Although threads are used, unfortunatelly it doesn't seem possible to speed up requests. Expect every request to take about 10 seconds to execute

In [ ]:
executor = ThreadPoolExecutor(max_workers=3)
insert_query(keywords_str, depths_str)

for i,contact in enumerate(contact_list):
    percent = int(100*i/len(contact_list))
    executor.submit(send_contact_request(contact, percent))
    